In [0]:
# !pip install scattertext
# !pip install -U spacy
# !python -m spacy download en_core_web_sm

In [0]:
# from flask import Flask, render_template, request, jsonify
import json
import warnings
import pandas as pd
import spacy
import scattertext as st
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
# from flask_cors import CORS
# from decouple import config
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)
nlp = spacy.load("en_core_web_sm")#if you run into problems here, 'Restart Runtime' and run all, it might fix things.
base_url = "https://www.yelp.com/biz/" 
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'Rc1lxc5lSKJYd162JHNMfQ'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data


def customtokensize(text):
    return re.findall("[\w']+", str(text))

df['tokenized_text'] = df[1].apply(customtokensize)
# stopwords = ['and','was','were','had','check-in','=','= =','u','want', 'u want', 'cuz','him',"i've",'on', 'her','told','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they', 'the', 'this','its', 'l','they','this',"don't",'the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',',']

stopwords = [',','"','!','-','&','?']

def filter_stopwords(text):
  nonstopwords = []
  for i in text:
    if i not in stopwords:
      nonstopwords.append(i)
  return nonstopwords
df['tokenized_text'] = df['tokenized_text'].apply(filter_stopwords)
df['parts_of_speech_reference'] = df['tokenized_text'].apply(filter_stopwords)
df['parts_of_speech_reference'] = df['parts_of_speech_reference'].str.join(' ')
# df['parts_of_speech_reference'] = df['tokenized_text'].str.join(' ')
df.head(1)

,0,1,2,tokenized_text,parts_of_speech_reference
0,\n 12/12/2019\n,I used to come here for dinner when I was traveling for work. Now I come here for brunch before Broadway matinees.,4.0 star rating,"[I, used, to, come, here, for, dinner, when, I, was, traveling, for, work, Now, I, come, here, for, brunch, before, Broadway, matinees]",I used to come here for dinner when I was traveling for work Now I come here for brunch before Broadway matinees


In [0]:
# def find_nounchunks(x):
#   """noun chunks greater than 2 without stopwords"""
#   noun_list = []
#   doc = nlp(str(x))
#   for chunk in doc.noun_chunks:
#     if len(chunk) > 2:
#       noun_list.append(chunk)
#   return noun_list

# df['nounchunk_list'] = df[1].apply(find_nounchunks)
# df[[1,'nounchunk_list']].head(1)

In [0]:
# def find_words_after_and_before_THE_A_AN(x):
#   """find_words_after_and_before_THE"""
#   total_lists = []
#   doc = nlp(str(x))
#   for token in range(len(doc)):
#     try:
#       word_list1 = []
#       if (doc[token].pos_ == 'PRON') or ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An'):
#         word_list1.append(doc[token])
#         word_list1.append(doc[token+1])
#         word_list1.append(doc[token+2])
#         word_list1.append(doc[token+3])
#         word_list1.append(doc[token+4])
#         word_list1.append(doc[token+5])
#         word_list1.append(doc[token+6])
#         word_list1.append(doc[token+7])
#         word_list1.append(doc[token+8])
#         word_list1.append(doc[token+9])
#         word_list1.append(doc[token+10])
#         word_list1.append(doc[token+11])
#       if len(word_list1) != 0:
#         total_lists.append(word_list1)
#     except IndexError or TypeError as e:
#       for token in range(len(doc)):
#         try:
#           word_list2 = []
#           if (doc[token].pos_ == 'PRON') or ((doc[token].pos_ == 'the') and (doc[token].pos_ == 'PRON')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An'):
#             word_list2.append(doc[token])
#             word_list2.append(doc[token+1])
#             word_list2.append(doc[token+2])
#             word_list2.append(doc[token+3])
#             word_list2.append(doc[token+4])
#             word_list2.append(doc[token+5])
#             word_list2.append(doc[token+6])
#             word_list2.append(doc[token+7])
#           if (len(word_list2) != 0) and (False for i in word_list2 for i in word_list1) and (False for i in word_list2 for i in total_lists):
#             total_lists.append(word_list2) 
#         except IndexError or TypeError as e:
#           for token in range(len(doc)):
#             try:
#               word_list3 = []
#               if ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')):
#                 word_list3.append(doc[token-1])
#                 word_list3.append(doc[token])
#                 word_list3.append(doc[token+1])
#                 word_list3.append(doc[token+2])
#               if (len(word_list3) != 0 ) and (False for i in word_list3 for i in word_list2) and (False for i in word_list3 for i in total_lists):
#                 total_lists.append(word_list3)
#             except IndexError or TypeError as e:
#               for token in range(len(doc)):
#                 try:
#                   word_list4 = []
#                   if (doc[token].text == 'the') or (doc[token].text == 'a') or (doc[token].text == 'an'):
#                     word_list4.append(doc[token])
#                     word_list4.append(doc[token+1])
#                     word_list4.append(doc[token+2])
#                     word_list4.append(doc[token+3])
#                   if (len(word_list4) != 0 ) and (False for i in word_list4 for i in word_list3) and (False for i in word_list4 for i in total_lists):
#                     total_lists.append(word_list4)
#                 except IndexError or TypeError as e:
#                   pass
#   return total_lists

# df['words_around_THE_A_AN'] = df['parts_of_speech_reference'].apply(find_words_after_and_before_THE_A_AN)
# df[[1,'words_around_THE_A_AN']].head(1)

In [0]:
# def find_phrases(x):
#   long_phrase_list = []
#   doc = nlp(str(x))
#   for token in range(len(doc)):
#     sub_list = []
#     try:
#       if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
#         if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list and doc[token+2] not in sub_list and doc[token+3] not in sub_list:
#           sub_list.append(doc[token-1])
#           sub_list.append(doc[token])
#           sub_list.append(doc[token+1])
#           sub_list.append(doc[token+2])
#           sub_list.append(doc[token+3])
#     except IndexError as e:
#       try:
#         if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
#           if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list and doc[token+2] not in sub_list:
#             sub_list.append(doc[token-1])
#             sub_list.append(doc[token])
#             sub_list.append(doc[token+1])
#             sub_list.append(doc[token+2])
#       except IndexError as e:
#         try:
#           if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
#             if doc[token-1] not in sub_list and doc[token] not in sub_list and doc[token+1] not in sub_list:
#               sub_list.append(doc[token-1])
#               sub_list.append(doc[token])
#               sub_list.append(doc[token+1])
#         except IndexError as e:
#           try:
#             if (doc[token].text == 'on') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
#               if doc[token] not in sub_list and doc[token+1] not in sub_list:
#                 sub_list.append(doc[token])
#                 sub_list.append(doc[token+1])
#           except IndexError as e:
#             try:
#               #captures words at the end of reviews
#               if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token-1].pos_ == 'ADJ'and doc[token].pos_ =='NOUN') or (doc[token-1].pos_ == 'VERB'and doc[token].pos_ =='NOUN')or (doc[token-1].pos_ == 'NOUN'and doc[token].pos_ =='NOUN'):
#                 if doc[token] not in sub_list and doc[token+1] not in sub_list:
#                   sub_list.append(doc[token-1])
#                   sub_list.append(doc[token])
#             except IndexError as e:
#               try:
#                 #captures words from the start of reviews
#                 if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'NOUN' and doc[token+1].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN')or (doc[token].pos_ == 'NOUN'and doc[token+1].pos_ =='NOUN'):
#                   if doc[token] not in sub_list and doc[token+1] not in sub_list:
#                     sub_list.append(doc[token])
#                     sub_list.append(doc[token+1])
#               except IndexError as e:
#                 pass
#     if (len(sub_list) != 0) and (sub_list not in long_phrase_list):
#       long_phrase_list.append(sub_list)
#   return long_phrase_list


# df['AdjNouns_NounNouns_Service_Food_For'] = df[1].apply(find_phrases)
# df[[1,'nounchunk_list','AdjNouns_NounNouns_Service_Food_For','words_around_THE_A_AN']].sample(1)

In [0]:
# def find_word_segments(x):
#   long_phrase_list = []
#   doc = nlp(str(x))
#   try:
#     for token in range(len(doc)):
#       sub_list = []
#       if (token < (int(len(doc))-5)) and ((doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN') or (doc[token].text == 'food') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN')):
#         sub_list.append(doc[token])
#         sub_list.append(doc[token+1])
#         sub_list.append(doc[token+2])
#         sub_list.append(doc[token+3])
#         sub_list.append(doc[token+4])
#         sub_list.append(doc[token+5])
#         # sub_list.append(doc[token+6])
#       if len(sub_list) != 0:
#         long_phrase_list.append(sub_list)
#   except IndexError as e:
#     pass
#   try:
#     for token in range(len(doc)):
#       sub_list = []
#       if (token < (int(len(doc))-5)) and ((doc[token].pos_ == 'PRON') or ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An')):
#         sub_list.append(doc[token])
#         sub_list.append(doc[token+1])
#         sub_list.append(doc[token+2])
#         sub_list.append(doc[token+3])
#         sub_list.append(doc[token+4])
#         sub_list.append(doc[token+5])
#       if len(sub_list) != 0:
#         long_phrase_list.append(sub_list)
#   except IndexError as e:
#     pass
#   return long_phrase_list

# df['word_segments'] = df[1].apply(find_word_segments)
# df[[1,'word_segments']].head(1)

In [0]:
def find_word_segment_combined(x):
  long_phrase_list = []
  doc = nlp(str(x))
  try:
    for token in range(len(doc)):
      sub_list = []
      if (token < (int(len(doc))-5)) and ((doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN') or (doc[token].text == 'food') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN')):
        sub_list.append(doc[token])
        sub_list.append(doc[token+1])
        sub_list.append(doc[token+2])
        sub_list.append(doc[token+3])
        sub_list.append(doc[token+4])
        sub_list.append(doc[token+5])
        sub_list.append(doc[token+6])
      if len(sub_list) != 0:
        long_phrase_list.append(sub_list)
  except IndexError as e:
    try:
      for token in range(len(doc)):
        sub_list = []
        if (token < (int(len(doc))-5)) and ((doc[token].pos_ == 'PRON') or ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An')):
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
          sub_list.append(doc[token+2])
          sub_list.append(doc[token+3])
          sub_list.append(doc[token+4])
          sub_list.append(doc[token+5])
        if len(sub_list) != 0:
          long_phrase_list.append(sub_list)
    except IndexError as e:
      try:
        word_list1 = []
        if (doc[token].pos_ == 'PRON') or ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An'):
          word_list1.append(doc[token])
          word_list1.append(doc[token+1])
          word_list1.append(doc[token+2])
          word_list1.append(doc[token+3])
          word_list1.append(doc[token+4])
          word_list1.append(doc[token+5])
          word_list1.append(doc[token+6])
          word_list1.append(doc[token+7])
          word_list1.append(doc[token+8])
          word_list1.append(doc[token+9])
          word_list1.append(doc[token+10])
          word_list1.append(doc[token+11])
        if len(word_list1) != 0:
          long_phrase_list.append(word_list1)
      except IndexError or TypeError as e:
        for token in range(len(doc)):
          try:
            word_list2 = []
            if (doc[token].pos_ == 'PRON') or ((doc[token].pos_ == 'the') and (doc[token].pos_ == 'PRON')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An'):
              word_list2.append(doc[token])
              word_list2.append(doc[token+1])
              word_list2.append(doc[token+2])
              word_list2.append(doc[token+3])
              word_list2.append(doc[token+4])
              word_list2.append(doc[token+5])
              word_list2.append(doc[token+6])
              word_list2.append(doc[token+7])
            if (len(word_list2) != 0) and (False for i in word_list2 for i in word_list1) and (False for i in word_list2 for i in long_phrase_list):
              long_phrase_list.append(word_list2) 
          except IndexError or TypeError as e:
            for token in range(len(doc)):
              try:
                word_list3 = []
                if ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')):
                  word_list3.append(doc[token-1])
                  word_list3.append(doc[token])
                  word_list3.append(doc[token+1])
                  word_list3.append(doc[token+2])
                if (len(word_list3) != 0 ) and (False for i in word_list3 for i in word_list2) and (False for i in word_list3 for i in long_phrase_list):
                  long_phrase_list.append(word_list3)
              except IndexError or TypeError as e:
                for token in range(len(doc)):
                  try:
                    word_list4 = []
                    if (doc[token].text == 'the') or (doc[token].text == 'a') or (doc[token].text == 'an'):
                      word_list4.append(doc[token])
                      word_list4.append(doc[token+1])
                      word_list4.append(doc[token+2])
                      word_list4.append(doc[token+3])
                    if (len(word_list4) != 0 ) and (False for i in word_list4 for i in word_list3) and (False for i in word_list4 for i in long_phrase_list):
                      long_phrase_list.append(word_list4)
                  except IndexError or TypeError as e:
                    try:
                      word_list5 = []
                      if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                        word_list5.append(doc[token-1])
                        word_list5.append(doc[token])
                        word_list5.append(doc[token+1])
                        word_list5.append(doc[token+2])
                        word_list5.append(doc[token+3])
                      if (len(word_list5) != 0) and (False for i in word_list5 for i in word_list4) and (False for i in word_list5 for i in long_phrase_list):
                        long_phrase_list.append(word_list5) 
                    except IndexError as e:
                      try:
                        word_list6 = []
                        if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                          word_list6.append(doc[token-1])
                          word_list6.append(doc[token])
                          word_list6.append(doc[token+1])
                          word_list6.append(doc[token+2])
                        if (len(word_list6) != 0) and (False for i in word_list6 for i in word_list5) and (False for i in word_list6 for i in long_phrase_list):
                          long_phrase_list.append(word_list6) 
                      except IndexError as e:
                        try:
                          word_list7 = []
                          if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                            word_list7.append(doc[token-1])
                            word_list7.append(doc[token])
                            word_list7.append(doc[token+1])
                          if (len(word_list7) != 0) and (False for i in word_list7 for i in word_list6) and (False for i in word_list7 for i in long_phrase_list):
                            long_phrase_list.append(word_list7) 
                        except IndexError as e:
                          try:
                            word_list8 = []
                            if (doc[token].text == 'on') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                              word_list8.append(doc[token])
                              word_list8.append(doc[token+1])
                            if (len(word_list8) != 0) and (False for i in word_list8 for i in word_list7) and (False for i in word_list8 for i in long_phrase_list):
                              long_phrase_list.append(word_list8) 
                          except IndexError as e:
                            try:
                              word_list9 = []
                              #captures words at the end of reviews
                              if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token-1].pos_ == 'ADJ'and doc[token].pos_ =='NOUN') or (doc[token-1].pos_ == 'VERB'and doc[token].pos_ =='NOUN')or (doc[token-1].pos_ == 'NOUN'and doc[token].pos_ =='NOUN'):
                                word_list9.append(doc[token-1])
                                word_list9.append(doc[token])
                              if (len(word_list9) != 0) and (False for i in word_list9 for i in word_list8) and (False for i in word_list9 for i in long_phrase_list):
                                long_phrase_list.append(word_list9) 
                            except IndexError as e:
                              try:
                                word_list10 = []
                                #captures words from the start of reviews
                                if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'NOUN' and doc[token+1].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN')or (doc[token].pos_ == 'NOUN'and doc[token+1].pos_ =='NOUN'):
                                  word_list10.append(doc[token])
                                  word_list10.append(doc[token+1])
                                if (len(word_list10) != 0) and (False for i in word_list10 for i in word_list9) and (False for i in word_list10 for i in long_phrase_list):
                                  long_phrase_list.append(word_list10) 
                              except IndexError as e:
                                pass
  return long_phrase_list

df['word_segments'] = df[1].apply(find_word_segment_combined)
df[[1,'word_segments']].head(3)

,1,word_segments
0,I used to come here for dinner when I was traveling for work. Now I come here for brunch before Broadway matinees.,[]
1,"Great for girls night, our waiter was amazing. Great at food and drink recommendations. The ricotta was delicious! Good dinner, however pricy!","[[food, and, drink, recommendations, ., The, ricotta], [drink, recommendations, ., The, ricotta, was, delicious], [The, ricotta, was, delicious, !, Good]]"
2,"I am an Italian food fanatic, so once I saw the beautiful decor and delicious menu, I figured I had to visit. The tomato soup was quite delicious, even though the thickness was more alike to a pasta sauce as it was also filled with cheese. For my main course, I ordered the Salmon. BEST DECISION EVER. Other members in my party ordered the spinach pasta and the gnocchi and they just simply did not match up to the salmon which bursted with flavor. For dessert, I would definitely recommend the Lemon Custard Tart!","[[Italian, food, fanatic, ,, so, once, I], [food, fanatic, ,, so, once, I, saw], [delicious, menu, ,, I, figured, I, had], [main, course, ,, I, ordered, the, Salmon], [ordered, the, Salmon, ., BEST, DECISION, EVER], [Other, members, in, my, party, ordered, the], [ordered, the, spinach, pasta, and, the, gnocchi], [not, match, up, to, the, salmon, which]]"


In [0]:
df['word_segments_unpacked'] = df['word_segments'].apply(lambda l: [item for sublist in l for item in sublist])
df['word_segments_unpacked'] = df['word_segments_unpacked'].astype(str)
df['word_segments_unpacked'] = df['word_segments_unpacked'].apply(lambda x: ''.join([str(i) for i in x]))
df.head(3)

,0,1,2,tokenized_text,parts_of_speech_reference,word_segments,word_segments_unpacked
0,\n 12/12/2019\n,I used to come here for dinner when I was traveling for work. Now I come here for brunch before Broadway matinees.,4.0 star rating,"[I, used, to, come, here, for, dinner, when, I, was, traveling, for, work, Now, I, come, here, for, brunch, before, Broadway, matinees]",I used to come here for dinner when I was traveling for work Now I come here for brunch before Broadway matinees,[],[]
1,\n 12/11/2019\n,"Great for girls night, our waiter was amazing. Great at food and drink recommendations. The ricotta was delicious! Good dinner, however pricy!",4.0 star rating,"[Great, for, girls, night, our, waiter, was, amazing, Great, at, food, and, drink, recommendations, The, ricotta, was, delicious, Good, dinner, however, pricy]",Great for girls night our waiter was amazing Great at food and drink recommendations The ricotta was delicious Good dinner however pricy,"[[food, and, drink, recommendations, ., The, ricotta], [drink, recommendations, ., The, ricotta, was, delicious], [The, ricotta, was, delicious, !, Good]]","[food, and, drink, recommendations, ., The, ricotta, drink, recommendations, ., The, ricotta, was, delicious, The, ricotta, was, delicious, !, Good]"
2,\n 12/11/2019\n,"I am an Italian food fanatic, so once I saw the beautiful decor and delicious menu, I figured I had to visit. The tomato soup was quite delicious, even though the thickness was more alike to a pasta sauce as it was also filled with cheese. For my main course, I ordered the Salmon. BEST DECISION EVER. Other members in my party ordered the spinach pasta and the gnocchi and they just simply did not match up to the salmon which bursted with flavor. For dessert, I would definitely recommend the Lemon Custard Tart!",4.0 star rating,"[I, am, an, Italian, food, fanatic, so, once, I, saw, the, beautiful, decor, and, delicious, menu, I, figured, I, had, to, visit, The, tomato, soup, was, quite, delicious, even, though, the, thickness, was, more, alike, to, a, pasta, sauce, as, it, was, also, filled, with, cheese, For, my, main, course, I, ordered, the, Salmon, BEST, DECISION, EVER, Other, members, in, my, party, ordered, the, spinach, pasta, and, the, gnocchi, and, they, just, simply, did, not, match, up, to, the, salmon, which, bursted, with, flavor, For, dessert, I, would, definitely, recommend, the, Lemon, Custard, Tart]",I am an Italian food fanatic so once I saw the beautiful decor and delicious menu I figured I had to visit The tomato soup was quite delicious even though the thickness was more alike to a pasta sauce as it was also filled with cheese For my main course I ordered the Salmon BEST DECISION EVER Other members in my party ordered the spinach pasta and the gnocchi and they just simply did not match up to the salmon which bursted with flavor For dessert I would definitely recommend the Lemon Custard Tart,"[[Italian, food, fanatic, ,, so, once, I], [food, fanatic, ,, so, once, I, saw], [delicious, menu, ,, I, figured, I, had], [main, course, ,, I, ordered, the, Salmon], [ordered, the, Salmon, ., BEST, DECISION, EVER], [Other, members, in, my, party, ordered, the], [ordered, the, spinach, pasta, and, the, gnocchi], [not, match, up, to, the, salmon, which]]","[Italian, food, fanatic, ,, so, once, I, food, fanatic, ,, so, once, I, saw, delicious, menu, ,, I, figured, I, had, main, course, ,, I, ordered, the, Salmon, ordered, the, Salmon, ., BEST, DECISION, EVER, Other, members, in, my, party, ordered, the, ordered, the, spinach, pasta, and, the, gnocchi, not, match, up, to, the, salmon, which]"


In [0]:
phrase_count = df[['word_segments_unpacked', 2]]
import numpy as np

s= phrase_count.apply(lambda x: pd.Series(x['word_segments_unpacked']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'word_segments_unpacked'

phrase_count = phrase_count.drop('word_segments_unpacked', axis=1).join(s)
phrase_count.head(1)

,2,word_segments_unpacked
0,4.0 star rating,[]


In [0]:
phrase_count = pd.DataFrame(df['word_segments_unpacked'].str.split(',').tolist(), index=df[2]).stack()
phrase_count = phrase_count.reset_index()[[0, 2]] # var1 variable is currently labeled 0
phrase_count.columns = ['word_segments_unpacked', 'ratings'] # renaming var1
phrase_count = phrase_count.reset_index(drop=False)

# x = ''
# replaceDict = {'to' : '', '!]' : '', '-' : '' }    
# stopwords = ['to','We','had','.',"'re",'it','or','he','she','we','us','how','went','no',"'d",'"','of','has','a','by','bit','thing','NaN','place',' ','on','so','ok','i','and','they','None','was','you',"'ve",'did',"'d",'be','and','but','is','as','a','b','c','d','e','f','g','u','it','!','&','you','has','-','None',':','and','had','was','him','so','my',' ','did','would','on','her','him','it','is','by','bit','thing','NaN','place','[',']','while','check-in','=','= =','want', 'good','husband', 'want','love','something','your','they','your','cuz','him',"i've",'her','told', '1 check', 'i"m', "it's",'they', ' the','the ',' ', 'this','its','they','this',"don't",'the',',', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.']
# def filter_stopwords(text):
#   for i in str(text):
#     if i not in stopwords:
#       return str(text)

# phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].map(filter_stopwords)
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace('[','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace(']','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace('-','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace('.','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace(' ','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.lower()
# phrase_count = phrase_count[phrase_count['word_segments_unpacked']!=i for i in stopwords]

stopwords = ['0','1','2','3','4','5','6','7','8','9','/','$',"'d","'ll","'m",'+','maybe','from','first','here','only','put','where','got','sure','definitely','food','yet','our','go','since','really','very','two',"n't",'with','if',"'s",'which','came','all','me','(',')','makes','make','were','immediately','get','been','ahead','also','that','one','have','see','what','to','we','had','.',"'re",'it','or','he','she','we','us','how','went','no','"','of','has','by','bit','thing','place','so','ok','and','they','none','was','you',"'ve",'did','be','and','but','is','as','&','you','has','-',':','and','had','was','him','so','my','did','would','her','him','it','is','by','bit','thing','place','[',']','while','check-in','=','= =','want', 'good','husband', 'want','love','something','your','they','your','cuz','him',"i've",'her','told', 'check', 'i"m', "it's",'they', 'this','its','they','this',"don't",'the',',', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.']
def filter_stopwords(text):
  for i in str(text):
    if i not in stopwords:
      return str(text)

phrase_count = phrase_count[~phrase_count['word_segments_unpacked'].isin(stopwords)]#if item in stopwords list partially matches, delete

#full matches
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='i']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='a']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='an']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='am']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='at']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='are']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='in']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='on']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='for']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='']
phrase_count.sample(1)

,index,word_segments_unpacked,ratings
1331,1331,fixe,3.0 star rating


In [0]:
#test values
phrase_count.iloc[85]['word_segments_unpacked']

'bottomless'

In [0]:
# phrase_count.tail(188)

In [0]:
phrase_count_pivot = pd.pivot_table(phrase_count, index='word_segments_unpacked', columns='ratings', aggfunc='count', fill_value=0)
phrase_count_pivot.sort_values(by=('index','1.0 star rating'), ascending=False)

index                                  \
ratings                1.0 star rating 2.0 star rating 3.0 star rating   
word_segments_unpacked                                                   
service                             13              11              13   
wait                                 8               1               1   
order                                6               3               1   
lukewarm                             4               0               0   
not                                  4              21              23   
...                                ...             ...             ...   
fresh                                0               0               3   
friend                               0               0               0   
friendly                             0               0               1   
friends                              0               0               1   
                                     0               1               0   

                                                        
ratings                4.0 star rating 5.0 star rating  
word_segments_unpacked                                  
service                             30              41  
wait                                 3               4  
order                                5               7  
lukewarm                             0               0  
not                                 16              13  
...                                ...             ...  
fresh                                1               2  
friend                               1               4  
friendly                             6               4  
friends                              4               2  
                                     0               0  

[1069 rows x 5 columns]

In [0]:
phrase_count_pivot.columns = [''.join(col).strip() for col in phrase_count_pivot.columns.values]#flatten index levels part 1
phrase_count_pivot = pd.DataFrame(phrase_count_pivot.to_records())#flatten index levels part 2

In [0]:

#not necessary
# phrase_count_pivot['index1.0 star rating'] = phrase_count_pivot['index1.0 star rating']*(-2)
# phrase_count_pivot['index2.0 star rating'] = phrase_count_pivot['index2.0 star rating']*(-1)
# phrase_count_pivot['index3.0 star rating'] = phrase_count_pivot['index3.0 star rating']*(-0.1)
# phrase_count_pivot['index4.0 star rating'] = phrase_count_pivot['index4.0 star rating']*(1)
# phrase_count_pivot['index5.0 star rating'] = phrase_count_pivot['index5.0 star rating']*(2)

phrase_count_pivot['score'] = phrase_count_pivot['index1.0 star rating'] + phrase_count_pivot['index2.0 star rating'] + phrase_count_pivot['index3.0 star rating'] + phrase_count_pivot['index4.0 star rating'] + phrase_count_pivot['index5.0 star rating']

phrase_count_pivot.sort_values(by=('index1.0 star rating'), ascending=False)


,word_segments_unpacked,index1.0 star rating,index2.0 star rating,index3.0 star rating,index4.0 star rating,index5.0 star rating,score
810,service,13,11,13,30,41,108
1009,wait,8,1,1,3,4,17
628,order,6,3,1,5,7,22
534,lukewarm,4,0,0,0,0,4
607,not,4,21,23,16,13,77
...,...,...,...,...,...,...,...
379,fresh,0,0,3,1,2,6
380,friend,0,0,0,1,4,5
381,friendly,0,0,1,6,4,11
382,friends,0,0,1,4,2,7


In [0]:
# phrase_count_pivot['sum'] = (phrase_count_pivot['sum']-phrase_count_pivot['sum'].min()/phrase_count_pivot['sum'].max()-phrase_count_pivot['sum'].min())/phrase_count_pivot['sum'].max()
phrase_count_pivot['score'] = phrase_count_pivot['score'].div(phrase_count_pivot['score'].max(), axis=0)
phrase_count_pivot['score'] = phrase_count_pivot['score'].round(decimals=4)
phrase_count_pivot = phrase_count_pivot.sort_values(by=('score'), ascending=False)
phrase_count_pivot.head(2)

,word_segments_unpacked,index1.0 star rating,index2.0 star rating,index3.0 star rating,index4.0 star rating,index5.0 star rating,score
407,great,4,6,5,30,82,1.0000
810,service,13,11,13,30,41,0.8504


In [0]:
phrase_count_pivot['word_segments_unpacked'].iloc[-2]

'lights'

In [0]:
#find where the worst terms appear in the review word segments and extract that word phrase to give context
worst_term = df[df['word_segments_unpacked'].str.contains(phrase_count_pivot['word_segments_unpacked'].iloc[-2])]
worst_term = worst_term.sort_values(by=2, ascending=True)
worst_term.head(1)

,0,1,2,tokenized_text,parts_of_speech_reference,word_segments,word_segments_unpacked
17,\n 11/2/2019\n,"Hells Kitchen is having a resurgence in good restaurants and Bea is for sure among them. A large restaurant/bar, my friend and I were seated in the ""indoor-outdoor"" area featuring trees with fairy lights and Edward Scissorhands playing silently on the wall behind us. The drink menu is well rounded and features a variety of creative cocktails, the food is high quality and we found our server to be friendly and helpful. It's the kind of oasis to escape to after a long day of working in the Times Square area- IF you can get a table.",5.0 star rating,"[Hells, Kitchen, is, having, a, resurgence, in, good, restaurants, and, Bea, is, for, sure, among, them, A, large, restaurant, bar, my, friend, and, I, were, seated, in, the, indoor, outdoor, area, featuring, trees, with, fairy, lights, and, Edward, Scissorhands, playing, silently, on, the, wall, behind, us, The, drink, menu, is, well, rounded, and, features, a, variety, of, creative, cocktails, the, food, is, high, quality, and, we, found, our, server, to, be, friendly, and, helpful, It's, the, kind, of, oasis, to, escape, to, after, a, long, day, of, working, in, the, Times, Square, area, IF, you, can, get, a, table]",Hells Kitchen is having a resurgence in good restaurants and Bea is for sure among them A large restaurant bar my friend and I were seated in the indoor outdoor area featuring trees with fairy lights and Edward Scissorhands playing silently on the wall behind us The drink menu is well rounded and features a variety of creative cocktails the food is high quality and we found our server to be friendly and helpful It's the kind of oasis to escape to after a long day of working in the Times Square area IF you can get a table,"[[good, restaurants, and, Bea, is, for, sure], [large, restaurant, /, bar, ,, my, friend], [featuring, trees, with, fairy, lights, and, Edward], [creative, cocktails, ,, the, food, is, high], [food, is, high, quality, and, we, found], [high, quality, and, we, found, our, server], [long, day, of, working, in, the, Times]]","[good, restaurants, and, Bea, is, for, sure, large, restaurant, /, bar, ,, my, friend, featuring, trees, with, fairy, lights, and, Edward, creative, cocktails, ,, the, food, is, high, food, is, high, quality, and, we, found, high, quality, and, we, found, our, server, long, day, of, working, in, the, Times]"


In [0]:
word_list_top_bottom = ''
for i in list(str(worst_term['word_segments_unpacked'])):
  word_list_top_bottom += i
for word in word_list_top_bottom:
  word_list = word_list_top_bottom.split(', ')
  for i in word_list:
    if phrase_count_pivot['word_segments_unpacked'].iloc[-2] == i:
      index = word_list.index(i)
      rejoined_word_list = ' '.join(word_list[max(0,index-2):min(index+5, len(word_list))])
print(rejoined_word_list)

with fairy lights and Edward creative cocktails
